In [1]:

import random
import scipy.stats as stats
import numpy
import copy
from scipy.stats import poisson
import math

def multinomCDF_log(G, k, p, tau_p):
    s = float(k);
    log_cdf = -poisson.logpmf(k,s);
    gamma1 = 0.0;
    gamma2 = 0.0;
    sum_s2 = 0.0;
    sum_mu = 0.0;
    
    # P(W=k)
    for i in range(0,G):
        sp = s*p[i];
        
        pcdf = poisson.cdf(tau_p[i],sp);
        log_cdf += numpy.log(pcdf);
        
        mu = sp*(1-poisson.pmf(tau_p[i],sp)/pcdf);
        s2 = mu-(tau_p[i]-mu)*(sp-mu);
        
        mr = tau_p[i];
        mf2 = sp*mu-mr*(sp-mu);
        
        mr *= tau_p[i]-1;
        mf3 = sp*mf2-mr*(sp-mu);
        
        mr *= tau_p[i]-2;
        mf4 = sp*mf3-mr*(sp-mu);
        
        mu2 = mf2+mu*(1-mu);
        mu3 = mf3+mf2*(3-3*mu)+mu*(1+mu*(-3+2*mu));
        mu4 = mf4+mf3*(6-4*mu)+mf2*(7+mu*(-12+6*mu))+mu*(1+mu*(-4+mu*(6-3*mu)));
        
        gamma1 += mu3;
        gamma2 += mu4-3*s2*s2;
        sum_mu += mu;
        sum_s2 += s2; 
    sp = numpy.sqrt(sum_s2);
    gamma1 /= sum_s2*sp;
    gamma2 /= sum_s2*sum_s2;
    
    x = (k-sum_mu)/sp;
    x2 = x*x;
    
    PWN = (-x2/2
    +numpy.log(1+gamma1/6*x*(x2-3)+gamma2/24*(x2*x2-6*x2+3)
    +gamma1*gamma1/72*(((x2-15)*x2+45)*x2-15))
    -numpy.log(2*math.pi)/2 -numpy.log(sp));
    
    log_cdf += PWN;
    return log_cdf;

def multinomCDF(G, k, p, tau_p):
    return numpy.exp(multinomCDF_log(G, k, p, tau_p ));

def multinomial_icdf_most_likely(G, k, p, a, tau):
    tau_p = [k] + list(tau);
    temp = copy.copy(tau_p)
    cdf = multinomCDF(G, k, p, tau_p)
    new_cdf = 0;
    initial = 1;
    not_fulfilled = 0;
    
    if(cdf > a):
        return tau_p;
    for i in range(len(tau_p)-1):
        temp[i+1] = temp[i+1]+1;
        if(initial == 1):
            tau_p = copy.copy(temp);
            cdf = multinomCDF(G, k, p, tau_p);
            initial = 0;
        else:
            new_cdf = multinomCDF(G, k, p, temp)
            if(new_cdf >= a and new_cdf >= cdf):
                tau_p = copy.copy(temp);
                cdf = multinomCDF(G, k, p, tau_p); 
        if(new_cdf >= a or cdf >= a):
            if(not_fulfilled == 1):
                tau_p = copy.copy(temp);
                not_fulfilled = 0;
            else:
                temp[i+1] = temp[i+1]-1
                not_fulfilled = 0
        else:
            not_fulfilled = 1
    return tau_p


def multinomial_icdf_most_unlikely(G, k, p, a, tau):
    tau_p = [k] + list(tau);
    temp = copy.copy(tau_p)
    cdf = multinomCDF(G, k, p, tau_p)
    new_cdf = 0;
    initial = 1;
    not_fulfilled = 0;
    
    if(cdf > a):
        return tau_p;
    for i in range(len(tau_p)-1):
        temp[i+1] = temp[i+1]+1;
        if(initial == 1):
            tau_p = copy.copy(temp);
            cdf = multinomCDF(G, k, p, tau_p);
            initial = 0;
        else:
            new_cdf = multinomCDF(G, k, p, temp)
            if(new_cdf >= a and new_cdf <= cdf):
                tau_p = copy.copy(temp);
                cdf = multinomCDF(G, k, p, tau_p); 
        if(new_cdf >= a or cdf >= a):
            if(not_fulfilled == 1):
                tau_p = copy.copy(temp);
                not_fulfilled = 0;
            else:
                temp[i+1] = temp[i+1]-1
                not_fulfilled = 0
        else:
            not_fulfilled = 1
    return tau_p

# def multinomial_icdf_between(G, k, p, tau, most_unlike, most_like):
#     tau_p = [k] + list(tau);
#     temp = copy.copy(tau_p)
#     cdf = multinomCDF(G, k, p, tau_p)
#     new_cdf = 0;
#     initial = 1;
#     not_fulfilled = 0;
#     like = numpy.append([k],most_like)
#     unlike = numpy.append([k],most_unlike)
# #     print like,", ",unlike
# #     print multinomCDF(G, k, p, like), ", ",multinomCDF(G, k, p, unlike)
#     like_cdf = multinomCDF(G, k, p, like)
#     unlike_cdf = multinomCDF(G, k, p, unlike)
# #     a = (multinomCDF(G, k, p, like)+multinomCDF(G, k, p, unlike))/2
# #     print a
    
#     if(cdf >= unlike_cdf and cdf <= like_cdf):
#         return tau_p;
#     for i in range(len(tau_p)-1):
#         temp[i+1] = temp[i+1]+1;
#         if(initial == 1):
#             tau_p = copy.copy(temp);
#             cdf = multinomCDF(G, k, p, tau_p);
#             initial = 0;
#         else:
#             new_cdf = multinomCDF(G, k, p, temp)
#             if(new_cdf > unlike_cdf and new_cdf <= like_cdf):
#                 print "tau_p ",tau_p
#                 print "temp", temp
#                 print "new_cdf",new_cdf
#                 print "like ", like_cdf
#                 print "unlike ", unlike_cdf
#                 tau_p = copy.copy(temp);
#                 cdf = multinomCDF(G, k, p, tau_p); 
#         if(new_cdf >= unlike_cdf or cdf >= unlike_cdf):
#             print "tauP if",tau_p
#             print "tau if", temp
#             if(not_fulfilled == 1):
#                 tau_p = copy.copy(temp);
#                 not_fulfilled = 0;
# #                 print "tauP",tau_p
# #                 print "tau", temp    
#             else:
#                 temp[i+1] = temp[i+1]-1
#                 not_fulfilled = 0
#         else:
#             not_fulfilled = 1
# #         print "tau_p end ",tau_p    
#     return tau_p

def multinomial_icdf_between(G, k, p, tau, alpha, most_unlike, most_like):
    tau_p = [k] + list(tau);
    temp = copy.copy(tau_p)
    cdf = multinomCDF(G, k, p, tau_p)
    new_cdf = 0;
    initial = 1;
    not_fulfilled = 0;
    like = numpy.append([k],most_like)
    unlike = numpy.append([k],most_unlike)
    
    if(multinomCDF(G, k, p, unlike)<multinomCDF(G, k, p,like)):
        a = multinomCDF(G, k, p, unlike)
        upper_a = multinomCDF(G, k, p, like)
    else:
        a = multinomCDF(G, k, p, like)
        upper_a = multinomCDF(G, k, p, unlike)
        
    if(cdf > alpha):
        return tau_p;
    for i in range(len(tau_p)-1):
        temp[i+1] = temp[i+1]+1;
        if(initial == 1):
            tau_p = copy.copy(temp);
            cdf = multinomCDF(G, k, p, tau_p);
            initial = 0;
        else:
            new_cdf = multinomCDF(G, k, p, temp)
            if(new_cdf > a and new_cdf <= upper_a):
                tau_p = copy.copy(temp);
                cdf = multinomCDF(G, k, p, tau_p); 
                return tau_p
                
#         print "a: ", a
#         print "upper1 : ", upper_a
#         print "cdf: ",cdf
#         print "new_cdf: ", new_cdf
#         print "tau_p",tau_p
#         print "temp",temp

        if((new_cdf >= a or cdf >= a) and (new_cdf <= upper_a or cdf <= upper_a)):
#             print "in1"
            if(not_fulfilled == 1):
                tau_p = copy.copy(temp);
                not_fulfilled = 0;
            else:
#                 print "IN"
                temp[i+1] = temp[i+1]-1
                not_fulfilled = 0
        else:
            not_fulfilled = 1
            if(new_cdf > upper_a or cdf > upper_a):
                temp[i+1] = temp[i+1]-1
    return tau_p

In [2]:
p = [0.4, 0.3, 0.2, 0.1];
a = 0.1;
k = 100;
positions = numpy.array(list(range(k))) + 1;

least_items_like = [];
least_items_unlike = [];
least_items_between = []

tau_like = numpy.zeros(len(p)-1);
tau_unlike = numpy.zeros(len(p)-1);
tau_between = numpy.zeros(len(p)-1);

for i in positions:
    tau_p_most_like = multinomial_icdf_most_likely(len(p), i, p , a, tau_like)[1:]
    least_items_like.append(numpy.array(tau_p_most_like));
    tau_like = copy.copy(tau_p_most_like);
    
    tau_p_most_unlike = multinomial_icdf_most_unlikely(len(p), i, p , a, tau_unlike)[1:]
    least_items_unlike.append(numpy.array(tau_p_most_unlike));
    tau_unlike = copy.copy(tau_p_most_unlike);

    
for i in positions:
    tau_p_between = multinomial_icdf_between(len(p), i, p , tau_between, a, least_items_unlike[i-1], least_items_like[i-1])[1:]
    least_items_between.append(numpy.array(tau_p_between));
    tau_between = copy.copy(tau_p_between);    
    
for i in range (k):
    print i+1,"  : ",least_items_between[i];
    test = numpy.append([i+1],least_items_between[i])
    print "CDF: ",multinomCDF(4, i+1, p, test)
    test = []
# print multinomCDF(4, 5, p, [4,0,2,2])
# multinomial_icdf_between(len(p), 5, p , [0,2,0], a, [0,2,1], [1,1,0])

1   :  [ 0.  0.  0.]
CDF:  0.489076210108
2   :  [ 0.  0.  0.]
CDF:  0.143143629273
3   :  [ 0.  1.  0.]
CDF:  0.162375286614
4   :  [ 0.  2.  0.]
CDF:  0.117164855603
5   :  [ 1.  2.  0.]
CDF:  0.24181878942
6   :  [ 1.  2.  0.]
CDF:  0.1473690501
7   :  [ 1.  3.  0.]
CDF:  0.114518668712
8   :  [ 2.  3.  0.]
CDF:  0.178004965616
9   :  [ 2.  3.  0.]
CDF:  0.11814652546
10   :  [ 2.  3.  1.]
CDF:  0.192637868503
11   :  [ 2.  3.  1.]
CDF:  0.128844071032
12   :  [ 2.  4.  1.]
CDF:  0.113875480374
13   :  [ 3.  4.  1.]
CDF:  0.180095512129
14   :  [ 3.  4.  1.]
CDF:  0.126763325702
15   :  [ 3.  4.  2.]
CDF:  0.150873323479
16   :  [ 3.  4.  2.]
CDF:  0.105744922951
17   :  [ 4.  4.  2.]
CDF:  0.159635639799
18   :  [ 4.  4.  2.]
CDF:  0.113520954434
19   :  [ 4.  5.  2.]
CDF:  0.114904361514
20   :  [ 4.  5.  3.]
CDF:  0.119801471128
21   :  [ 5.  5.  3.]
CDF:  0.176701203128
22   :  [ 5.  5.  3.]
CDF:  0.131953646382
23   :  [ 5.  6.  3.]
CDF:  0.133462248083
24   :  [ 6.  6.  3.]
CD

In [3]:
for i in range (k):
    print i+1,"  : ",least_items_unlike[i];
    test = numpy.append([i+1],least_items_unlike[i])
    print "CDF: ",multinomCDF(4, i+1, p, test)
    test = []

1   :  [ 0.  0.  0.]
CDF:  0.489076210108
2   :  [ 0.  0.  0.]
CDF:  0.143143629273
3   :  [ 0.  0.  1.]
CDF:  0.108139163863
4   :  [ 0.  1.  1.]
CDF:  0.144192159119
5   :  [ 0.  2.  1.]
CDF:  0.122199359253
6   :  [ 1.  2.  1.]
CDF:  0.291224823102
7   :  [ 1.  2.  1.]
CDF:  0.188335431135
8   :  [ 1.  2.  1.]
CDF:  0.115954125076
9   :  [ 1.  3.  1.]
CDF:  0.104072282244
10   :  [ 2.  3.  1.]
CDF:  0.192637868503
11   :  [ 2.  3.  1.]
CDF:  0.128844071032
12   :  [ 2.  4.  1.]
CDF:  0.113875480374
13   :  [ 2.  4.  2.]
CDF:  0.123882635041
14   :  [ 2.  5.  2.]
CDF:  0.105066105608
15   :  [ 3.  5.  2.]
CDF:  0.188294602112
16   :  [ 3.  5.  2.]
CDF:  0.138532117453
17   :  [ 3.  6.  2.]
CDF:  0.117859398753
18   :  [ 3.  6.  3.]
CDF:  0.117893667499
19   :  [ 3.  6.  4.]
CDF:  0.102391018434
20   :  [ 4.  6.  4.]
CDF:  0.179443593253
21   :  [ 4.  6.  4.]
CDF:  0.138434048158
22   :  [ 4.  6.  4.]
CDF:  0.104725963546
23   :  [ 5.  6.  4.]
CDF:  0.167221419331
24   :  [ 5.  6.  4.

In [6]:
multinomCDF(4, 20, [0.6,0.15,0.15,0.1], [20,5,4,2])

0.50509949449866642